In [136]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

## Group Assignment
### Team Number: 11
### Team Member Names: Akram, Annie, Jester
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [137]:
# Important Constants: 
amount = 1_000_000 # Initial investment amount of $1,000,000
group = 11

# Reading in CSV file: 
tickers = pd.read_csv('Tickers.csv')
ticker_lst = list(tickers['Tickers'])

# Initializing variable to store the tickers we will use in our portfolio
columns = ['Ticker', 'Price', 'Currency', 'Shares', 'Value', 'Weight']
Portfolio_Final = pd.DataFrame(columns=columns)

In [138]:
# filtering stocks
valid_tickers = []
invalid_tickers = []

# loop through all tickers to check
for ticker in ticker_lst:
    try:
        info = yf.Ticker(ticker).fast_info
        if (info and info.get("exchange") is not None): # filter for stocks delisted on yfinance
            if(info.get("currency") == "USD" or info.get("currency") == "CAD"): # filter for stocks that are not USD
                valid_tickers.append(ticker)
            else:
                invalid_tickers.append(ticker)
    except:
        invalid_tickers.append(ticker)

print("The following tickers were removed:", invalid_tickers)

print("")

$AGN: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


The following tickers were removed: ['AGN', 'CELG', 'MON', 'RTN']



In [158]:
# sharpe ratio optimization

def optimal_weights(tickers, start_date, end_date, risk_free_rate):
    # download data
    data = yf.download(tickers, start=start_date, end=end_date)['Close']

    # calculate mean return of stocks and covariance of stocks
    returns = data.pct_change()
    returns.drop(index=returns.index[0], inplace = True)
    mean_returns = returns.mean()
    covariance_matrix = returns.cov()

    def neg_sharpe(weights):
        # calculate portfolio expected return by weighing each stock's expected return
        portfolio_expected_return = np.sum(weights*mean_returns)    

        portfolio_variance = 0
        # calculate portfolio risk (std) by finding the portfolio variance, which is affected by covariance
        for i in range(len(weights)):
            for j in range(len(weights)):
                portfolio_variance += weights[i] * weights[j] * covariance_matrix.iloc[i, j]
        portfolio_std = np.sqrt(portfolio_variance)
        
        # calculate sharpe ratio
        sharpe = (portfolio_expected_return - risk_free_rate)/portfolio_std
        
        return -sharpe #make sharpe ratio negative for minimize function


    # constraints
    def check_sum(weights): 
        return np.sum(weights)-1 #returns 0 if weights sum up to 1
    constraints = {'type': 'eq', 'fun': check_sum}

    min_weight = 1/(2*len(tickers))
    max_weight = 0.4

    bounds = [(min_weight, max_weight)]*len(tickers)

    # initial guess
    init_guess = [1.0/len(tickers)]*len(tickers)

    results = minimize(neg_sharpe, init_guess, method="SLSQP", bounds=bounds, constraints=constraints).x

    return results


tickers = ["AAPL", "MSFT", "GOOGL", "AMZN"]

print(optimal_weights(tickers, "2023-01-01", "2024-01-01", 0.02))


[*********************100%***********************]  4 of 4 completed


[0.125 0.4   0.35  0.125]


In [141]:
# Code to output final dataframe to a CSV file called Stocks_Group_XX.csv
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv(f'Stocks_Group_{group}.csv', index=False)

In [8]:
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

---
<p style="color: #004dd3">
Akram Jamil
</p>

<p style="color: #2C8CA9">
Jester Yang
</p>

<p style="color: #3cc19d;">
Annie Wong
</p>

---